In [1]:
# Importamos librería Pandas

import pandas as pd

In [2]:
# Leemos el dataset

complete = pd.read_csv("../data/f18/complete.csv", encoding = 'utf8')
complete.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera División,1985-02-05,185.0,...,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera División,1987-06-24,170.0,...,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,False,False,False,False,False,False,False,False,False,False
3,176580,L. Suárez,Luis Suárez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera División,1987-01-24,182.0,...,False,False,False,False,False,False,False,False,False,False
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,...,False,False,False,False,False,False,False,False,False,True


In [3]:
# Computamos la posición

def position(df):
  if (df["prefers_gk"] == True):
    return 'gk'
  elif (df["prefers_cb"] == True) | (df["prefers_lcb"] == True) | (df["prefers_rcb"] == True):
    return 'cb'
  elif (df["prefers_rb"] == True) | (df["prefers_lb"] == True) | (df["prefers_rwb"] == True) | \
  (df["prefers_lwb"] == True):
    return 'wb'
  elif (df["prefers_cdm"] == True) | (df["prefers_rdm"] == True) | (df["prefers_ldm"] == True) | \
  (df["prefers_cm"] == True) | (df["prefers_rcm"] == True) | (df["prefers_lcm"] == True) | \
  (df["prefers_lm"] == True) | (df["prefers_rm"] == True):
    return 'mid'
  elif (df["prefers_lam"] == True) | (df["prefers_ram"] == True) | (df["prefers_cam"] == True):
    return "f_mid"
  elif (df["prefers_lw"] == True) | (df["prefers_rw"] == True):
    return 'wing'
  elif (df["prefers_st"] == True) | (df["prefers_cf"] == True) | (df["prefers_lf"] == True) | \
  (df["prefers_rf"] == True) | (df["prefers_ls"] == True) | (df["prefers_rs"] == True):
    return 'st'
  else:
    return 'oth'

complete['position_feat'] = complete.apply(position, axis=1)

In [4]:
# Obtenemos las columnas que nos interesan

columns = ["ID", "position_feat", "acceleration", "sprint_speed", "sho", "heading_accuracy", "pas", "ball_control", "dribbling", "agility", 
           "def", "composure", "stamina", "strength", "gk_diving", "gk_handling", "gk_kicking", "gk_positioning","gk_reflexes",
           "jumping"]
df = complete[columns].copy()
df.head()

,ID,position_feat,acceleration,sprint_speed,sho,heading_accuracy,pas,ball_control,dribbling,agility,def,composure,stamina,strength,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,jumping
0,20801,wing,89,91,93,88,82,93,91,89,33,95,92,80,7,11,15,14,11,95
1,158023,wing,92,87,90,71,86,95,97,90,26,96,73,59,6,11,15,14,8,68
2,190871,wing,94,90,84,62,79,95,96,96,30,92,78,53,9,9,15,15,11,61
3,176580,st,88,77,90,77,79,91,86,86,42,83,89,80,27,25,31,33,37,69
4,167495,gk,58,61,90,25,95,48,30,52,60,70,44,83,91,90,95,91,89,78


In [5]:
#Despivotamos los valores con melt

dfMelt = pd.melt(df, id_vars=["ID", "position_feat"], value_vars=["acceleration", "sprint_speed", "sho", "heading_accuracy", "pas", "ball_control", "dribbling", "agility", 
           "def", "composure", "stamina", "strength", "gk_diving", "gk_handling", "gk_kicking", "gk_positioning","gk_reflexes",
           "jumping"], var_name="feature", value_name="feature_value")
dfMelt.head()

,ID,position_feat,feature,feature_value
0,20801,wing,acceleration,89
1,158023,wing,acceleration,92
2,190871,wing,acceleration,94
3,176580,st,acceleration,88
4,167495,gk,acceleration,58


In [6]:
#Eliminamos las filas no válidas (por ejemplo, caracteristicas de porteros para jugadoers de campo)

dfMelt = dfMelt.drop(dfMelt[(dfMelt.position_feat == "gk") & ((dfMelt.feature == "sho") | (dfMelt.feature == "heading_accuracy") |
                            (dfMelt.feature == "dribbling") | (dfMelt.feature == "ball_control") |
                            (dfMelt.feature == "def") | (dfMelt.feature == "acceleration"))].index)

dfMelt = dfMelt.drop(dfMelt[(dfMelt.position_feat != "gk") & ((dfMelt.feature == "gk_diving") | (dfMelt.feature == "gk_handling") |
                            (dfMelt.feature == "gk_kicking") | (dfMelt.feature == "gk_positioning") |
                            (dfMelt.feature == "gk_reflexes") | (dfMelt.feature == "jumping"))].index)

dfMelt.head()

,ID,position_feat,feature,feature_value
0,20801,wing,acceleration,89
1,158023,wing,acceleration,92
2,190871,wing,acceleration,94
3,176580,st,acceleration,88
5,188545,st,acceleration,79


In [7]:
# Escribimos el resultado en CSV

dfMelt.to_csv('../data/output/features_2018.csv', sep=',', encoding='utf-8')